In [181]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn import preprocessing, linear_model
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

%matplotlib inline

Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке! Вам понадобится много признаков. Некоторые из них у вас уже есть — это:
<ol>
<li>идентификатор географической зоны
<li>дата и время
<li>количество поездок в периоды, предшествующие прогнозируемому
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
<li>Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!
</ol>



Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования y^T+1|T, другая для y^T+2|T и т.д.

<ol>Чтобы сдать задание, выполните следующую последовательность действий.
<li>Для каждой из шести задач прогнозирования y^T+i|T,i=1,…,6 сформируйте выборки. Откликом будет yT+i при всевозможных значениях T, а признаки можно использовать следующие:
<ul>
<li>идентификатор географической зоны — категориальный
<li>год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так (done)
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA (done)
<li>сами значения прогнозов ARIMA y^T+i|TARIMA
<li>количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li>количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
</ul>
Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании y^T+i|T,i=1,…,6 вы можете учитывать только значения y до момента времени T включительно.

<li>Разбейте каждую из шести выборок на три части:

    обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
    тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
    итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016

<li>Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:

    допускала попарные взаимодействия между признаками
    была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)

<li>Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:
Qmay=1R∗739∗6∑r=1R∑T=2016.04.3023:002016.05.3117:00∑i=16y^T|T+ir−yT+ir.
Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу Q, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

<li>Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

<li>Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

<li>Загрузите ноутбук в форму.

Подгружаем данные

In [182]:
# id нужных регионов
regsDf = pd.read_csv('../crowdRegs.csv',names=['id','regId']);  

# временные ряды для этих регионов
df = pd.read_pickle('../loadData/crowdRegs3.pcl')
df.columns = regsDf.regId.values.astype('str')

In [183]:
df.head()

,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2014-01-01 00:00:00,87,146,70,113,367,645,589,799,948,321,...,9,0,5,89,10,35,9,106,22,71
2014-01-01 01:00:00,92,184,93,153,539,604,490,635,667,225,...,24,0,3,22,2,5,0,87,0,44
2014-01-01 02:00:00,108,165,55,151,443,571,465,499,455,124,...,27,0,3,23,1,1,0,39,0,1
2014-01-01 03:00:00,77,108,32,112,372,533,442,370,307,101,...,57,0,0,3,2,1,0,5,1,0
2014-01-01 04:00:00,47,79,22,77,213,383,296,319,261,87,...,38,0,1,9,1,8,0,29,1,18


In [184]:
def getCommonFeature(Kw=5,Ka=2):
    #parameters:
    #  Kw is number of weeks harmonics
    #  Ka is number of annual harmonics

    df2 = pd.DataFrame(index=index1)

    df2 = df2.assign(linear = (df2.index - datetime.datetime(2014,1,1,0,0,0))/np.timedelta64(1, 'h'))

    # час — эти признаки можно пробовать брать и категориальными
    # и непрерывными, можно даже и так, и так

    # добавляем гармонические фичи
    for ind in range(1,Kw+1):
        df2['weekCos'+str(ind)]= np.cos(np.pi*df2.linear*ind/168);
        df2['weekSin'+str(ind)]= np.sin(np.pi*df2.linear*ind/168);
    
    for ind in range(1,Ka+1):
        df2['yearCos'+str(ind)]= np.cos(2*np.pi*df2.linear*ind/8766);
        df2['yearSin'+str(ind)]= np.sin(2*np.pi*df2.linear*ind/8766);

    # добавляем числовое и категориальные свойства для дней недели
    df2 = df2.assign(dayOfWeek = df2.index.dayofweek)
    lbDays = preprocessing.LabelBinarizer()
    lbDays.fit(list(np.arange(6)))
    DoW = pd.DataFrame(lbDays.transform(df2.index.dayofweek),columns = ['dayOfWeek_'+str(x) for x in np.arange(6)],
               index = df2.index)      
    df2 = df2.merge(DoW,left_index=True,right_index=True)

    # добавляем dummy variables для месяца
    df2 = df2.assign(month = df2.index.month)
    lbMonths = preprocessing.LabelBinarizer()
    lbMonths.fit(list(np.arange(12)))
    Months = pd.DataFrame(lbMonths.transform(df2.index.month),columns = ['month_'+str(x) for x in np.arange(1,13)],
                  index = df2.index)      
    df2 = df2.merge(Months,left_index=True,right_index=True);

    # добавляем год (вещественный)
    df2 = df2.assign(year = df.index.year)

    # добавляем день месяца (вещественный)
    df2 = df2.assign(day = df.index.day)

    # добавляем час (вещественный и категориальный)
    df2 = df2.assign(hour = df.index.hour)
    lbHours = preprocessing.LabelBinarizer()
    lbHours.fit(list(np.arange(24)))
    Hours = pd.DataFrame(lbHours.transform(df2.index.hour),columns = ['hour_'+str(x) for x in np.arange(24)],
               index = df2.index)      
    df2 = df2.merge(Hours,left_index=True,right_index=True)
    return df2

In [185]:
def processSeries(df,Kh = 6, Kp = 2):
    """
    Обработка одного данного ряда 
    parameters:
        df - начальный датафрейм, из которого выберем для обработки один ряд
        tReg - название ряда, который надо обработать
        Kh - количество отслеживаемых прошлых суточных лагов "назад"
        Kp - количество отслеживаемых прошлых периодических лагов (период 24 часа)

    """
 

    tDf = df.copy()
    for timeLag in np.arange(1,Kh+1):
        name = 'hourLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag)

    for timeLag in np.arange(1,Kp+1):
        name = 'periodicLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag*24)

    tDf.fillna(0,inplace=True)    

    # суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
    tDf.loc[:,'sum12'] = tDf.y.rolling(window = 12, min_periods = 1).sum()
    tDf.loc[:,'sum24'] = tDf.y.rolling(window = 24, min_periods = 1).sum()
    tDf.loc[:,'sumWeek'] = tDf.y.rolling(window = 168, min_periods = 1).sum()
    tDf.loc[:,'sumMonth'] = tDf.y.rolling(window = 720, min_periods = 1).sum()
    
    tDf.loc[:,'max12'] = tDf.y.rolling(window = 12, min_periods = 1).max()
    tDf.loc[:,'max24'] = tDf.y.rolling(window = 24, min_periods = 1).max()
    tDf.loc[:,'maxWeek'] = tDf.y.rolling(window = 168, min_periods = 1).max()
    tDf.loc[:,'maxMonth'] = tDf.y.rolling(window = 720, min_periods = 1).max()
    
    
    tDf.loc[:,'min12'] = tDf.y.rolling(window = 12, min_periods = 1).min()
    tDf.loc[:,'min24'] = tDf.y.rolling(window = 24, min_periods = 1).min()
    tDf.loc[:,'minWeek'] = tDf.y.rolling(window = 168, min_periods = 1).min()
    tDf.loc[:,'minMonth'] = tDf.y.rolling(window = 720, min_periods = 1).min()
    
    
    
    return tDf

In [186]:
# create multyindex dataframe
index1 = df.index # time is first index
index2 = df.columns.values.astype(int)

mIndex = pd.MultiIndex.from_product([index2, index1],names=['region','date'])
df_test = pd.DataFrame(columns=['y'],index = mIndex)

In [187]:
idx = pd.IndexSlice

In [188]:
# create MultiIndex dataframe with target variables    
for k in index2:
    pos = idx[k,:]
    df_test.loc[idx[k,:],'y'] = df.loc[:,str(k)].values
    # create target variables
    df_test.loc[pos,'y1'] = df_test.loc[pos,'y'].shift(-1).fillna(0)
    df_test.loc[pos,'y2'] = df_test.loc[pos,'y'].shift(-2).fillna(0)
    df_test.loc[pos,'y3'] = df_test.loc[pos,'y'].shift(-3).fillna(0)
    df_test.loc[pos,'y4'] = df_test.loc[pos,'y'].shift(-4).fillna(0)
    df_test.loc[pos,'y5'] = df_test.loc[pos,'y'].shift(-5).fillna(0)
    df_test.loc[pos,'y6'] = df_test.loc[pos,'y'].shift(-6).fillna(0)
    
df_test.head()    

y     y1     y2    y3    y4    y5    y6
region date                                                          
1075   2014-01-01 00:00:00   87   92.0  108.0  77.0  47.0  22.0  10.0
       2014-01-01 01:00:00   92  108.0   77.0  47.0  22.0  10.0  18.0
       2014-01-01 02:00:00  108   77.0   47.0  22.0  10.0  18.0  19.0
       2014-01-01 03:00:00   77   47.0   22.0  10.0  18.0  19.0  28.0
       2014-01-01 04:00:00   47   22.0   10.0  18.0  19.0  28.0  37.0

In [ ]:
# create common features
commonFeatures = getCommonFeature()
commonFeatures.index.name = 'date'
commonFeatures.head()

,linear,weekCos1,weekSin1,weekCos2,weekSin2,weekCos3,weekSin3,weekCos4,weekSin4,weekCos5,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,0.0,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
2014-01-01 01:00:00,1.0,0.999825,0.018699,0.999301,0.037391,0.998427,0.056070,0.997204,0.074730,0.995632,...,0,0,0,0,0,0,0,0,0,0
2014-01-01 02:00:00,2.0,0.999301,0.037391,0.997204,0.074730,0.993712,0.111964,0.988831,0.149042,0.982566,...,0,0,0,0,0,0,0,0,0,0
2014-01-01 03:00:00,3.0,0.998427,0.056070,0.993712,0.111964,0.985871,0.167506,0.974928,0.222521,0.960917,...,0,0,0,0,0,0,0,0,0,0
2014-01-01 04:00:00,4.0,0.997204,0.074730,0.988831,0.149042,0.974928,0.222521,0.955573,0.294755,0.930874,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# join common features and add unique ones for each subgroup 
final = pd.DataFrame()
for k in index2:
    # create tatget variables
    pos = idx[k,:]
    tmp = processSeries(df_test.loc[pos,:].join(commonFeatures))
    final = pd.concat([final,tmp])

In [ ]:
final.head()

In [ ]:
idx = pd.IndexSlice

In [ ]:
dropList = ['y1','y2','y3','y4','y5','y6','y','date','region']
targetList= ['y1','y2','y3','y4','y5','y6']

In [ ]:
# dataframe to save prediction
resIndex = pd.MultiIndex.from_product([final.index.levels[0],pd.date_range(start=startTest,end=endTest,freq='1H')],
                                    names=['region','date'])
df_res = pd.DataFrame(columns=['y1','y1_pr','y2','y2_pr','y3','y3_pr','y4','y4_pr','y5','y5_pr','y6','y6_pr'],index = resIndex)
saveDict = {'y1':'y1_pr','y2':'y2_pr','y3':'y3_pr','y4':'y4_pr','y5':'y5_pr','y6':'y6_pr'}

In [ ]:
# define time frames
startTrain = '2014-01-01 00:00:00'
endTrain   = '2016-04-30 23:00:00'

startValidation = '2016-05-01 00:00:00'
endValidation   = '2016-05-31 23:00:00'

startTest = '2016-06-01 00:00:00'
endTest   = '2016-06-30 23:00:00'

In [ ]:
for reg in final.index.levels[0].values:
    print reg
    train = final.loc[idx[reg,startTrain:endValidation],:].reset_index()
    test  = final.loc[idx[reg,startTest:endTest],:].reset_index()
    for target in targetList:
        print target
        #dropList = [x for x in targetList if x !=target]
        #dropList.append('y')
        linReg.fit(train.drop(dropList,axis=1),train.loc[:,target])
        prediction = linReg.predict(test.drop(dropList,axis=1))
        prediction[prediction<0] = 0
        print MAE(prediction,test.loc[:,target])
        #save results
        df_res.loc[idx[reg,:], target] = test.loc[:,target].values
        df_res.loc[idx[reg,:], saveDict.get(target)] = prediction
df_res.head()        

In [ ]:
calcMAE(df_res)

In [157]:
saveResults(df_res,fName='30_2.csv',delta=0)

Скор на LB равен 36.76791
Занулил отрицательные предсказания. Скор 36.67943

In [111]:
def saveResults(rdf, fName, delta = 30):
    rnd = np.round

    f = open(fName,'w')
    f.writelines('id,y\n')

    for ind, row in rdf.reset_index().iterrows():
        historyStart = row.date - datetime.timedelta(hours = 1)

        if historyStart > datetime.datetime(2016,6,30,17):
            continue

        s0 = str(row.region)+'_'+ str(datetime.datetime.strftime(historyStart, "%Y-%m-%d"))+ '_'+ str(historyStart.hour)

        s1 = s0 +'_1,'+str(rnd(row.get('y1_pr'))+delta) + '\n'
        f.writelines(s1)

        s2 = s0 +'_2,'+str(rnd(row.get('y2_pr'))+delta) + '\n'
        f.writelines(s2)

        s3 = s0 +'_3,'+str(rnd(row.get('y3_pr'))+delta) + '\n'
        f.writelines(s3)

        s4 = s0 +'_4,'+str(rnd(row.get('y4_pr'))+delta) + '\n'
        f.writelines(s4)

        s5 = s0 +'_5,'+str(rnd(row.get('y5_pr'))+delta) + '\n'
        f.writelines(s5)

        s6 = s0 +'_6,'+str(rnd(row.get('y6_pr'))+delta) + '\n'
        f.writelines(s6)

    f.close()    

In [152]:
def calcMAE(inpDf):
    score = 0
    testDf = inpDf.reset_index()
    for k, v in saveDict.iteritems():
        score += MAE(testDf.loc[:,k],testDf.loc[:,v])
    score = score / 6.0    
    print score
    

In [156]:
calcMAE(df_res)

26.408985497


In [161]:
dfJune = pd.read_pickle('../checkData/June2016.pcl')
dfJune.head()

,y,region,y1,y2,y3,y4,y5,y6
2016-06-01 00:00:00,16,1075,16.0,12.0,3.0,0.0,3.0,16.0
2016-06-01 01:00:00,16,1075,12.0,3.0,0.0,3.0,16.0,25.0
2016-06-01 02:00:00,12,1075,3.0,0.0,3.0,16.0,25.0,55.0
2016-06-01 03:00:00,3,1075,0.0,3.0,16.0,25.0,55.0,68.0
2016-06-01 04:00:00,0,1075,3.0,16.0,25.0,55.0,68.0,49.0


In [167]:
dfJune = dfJune.reset_index().rename(columns={'index':'date'}).set_index(['region','date'])
dfJune.head()

y    y1    y2    y3    y4    y5    y6
region date                                                       
1075   2016-06-01 00:00:00  16  16.0  12.0   3.0   0.0   3.0  16.0
       2016-06-01 01:00:00  16  12.0   3.0   0.0   3.0  16.0  25.0
       2016-06-01 02:00:00  12   3.0   0.0   3.0  16.0  25.0  55.0
       2016-06-01 03:00:00   3   0.0   3.0  16.0  25.0  55.0  68.0
       2016-06-01 04:00:00   0   3.0  16.0  25.0  55.0  68.0  49.0

In [170]:
# join common features and add unique ones for each subgroup 
finalJune = pd.DataFrame()
for k in index2:
    # create tatget variables
    pos = idx[k,:]
    tmp = processSeries(dfJune.loc[pos,:].join(commonFeatures.loc['06-2016']))
    finalJune = pd.concat([finalJune,tmp])

In [176]:
for reg in final.index.levels[0].values:
    print reg
    train = final.loc[idx[reg,startTrain:endValidation],:].reset_index()
    test  = finalJune.loc[idx[reg,startTest:endTest],:].reset_index()
    for target in targetList:
        print target
        #dropList = [x for x in targetList if x !=target]
        #dropList.append('y')
        linReg.fit(train.drop(dropList,axis=1),train.loc[:,target])
        prediction = linReg.predict(test.drop(dropList,axis=1))
        prediction[prediction<0] = 0
        print MAE(prediction,test.loc[:,target])
        #save results
        df_res.loc[idx[reg,:], target] = test.loc[:,target].values
        df_res.loc[idx[reg,:], saveDict.get(target)] = prediction
df_res.head()        

1075
y1
15.1405085005
y2
16.5878671462
y3
17.4756596647
y4
17.8462044597
y5
17.8788309701
y6
18.0039281408
1076
y1
23.3364753259
y2
24.1912818236
y3
24.5130871846
y4
24.6720015474
y5
24.710506964
y6
25.1327576321
1077
y1
19.9586673226
y2
20.8741828381
y3
21.348322989
y4
21.0256746179
y5
20.7194692545
y6
20.4561362905
1125
y1
12.8061633499
y2
13.4098420331
y3
13.7930015333
y4
13.8387837311
y5
13.6979949644
y6
13.7537739978
1126
y1
28.7836074043
y2
29.7237896039
y3
30.7757501243
y4
32.0448856222
y5
32.8380935188
y6
32.473294323
1127
y1
42.4989107254
y2
47.2976012709
y3
49.5150744682
y4
49.4202119819
y5
47.6224590386
y6
47.5293175796
1128
y1
50.5689726186
y2
57.3054623843
y3
60.5373130507
y4
60.6273802059
y5
59.4264954943
y6
58.6575634163
1129
y1
63.2900210762
y2
74.0285503881
y3
77.9430954798
y4
73.9531482503
y5
70.3269319698
y6
69.8819873707
1130
y1
79.7253604271
y2
91.3138362674
y3
97.8292452553
y4
93.6598655531
y5
86.1436609275
y6
84.515866206
1131
y1
32.5847510095
y2
36.7515020513
y3

y1    y1_pr    y2    y2_pr    y3    y3_pr    y4  \
region date                                                                     
1075   2016-06-01 00:00:00  16.0  3.25548  12.0  6.02036   3.0   8.9892   0.0   
       2016-06-01 01:00:00  12.0  9.24819   3.0  10.4773   0.0  13.1066   3.0   
       2016-06-01 02:00:00   3.0   10.849   0.0  12.1857   3.0  13.8932  16.0   
       2016-06-01 03:00:00   0.0  11.5391   3.0  12.2334  16.0   18.417  25.0   
       2016-06-01 04:00:00   3.0  6.60129  16.0   14.326  25.0  25.8311  55.0   

                              y4_pr    y5    y5_pr    y6    y6_pr  
region date                                                        
1075   2016-06-01 00:00:00  13.8707   3.0  19.6937  16.0  30.6543  
       2016-06-01 01:00:00  16.7964  16.0  26.4724  25.0  39.1153  
       2016-06-01 02:00:00  21.7251  25.0   33.005  55.0  50.8407  
       2016-06-01 03:00:00  28.2851  55.0  44.8531  68.0  39.1584  
       2016-06-01 04:00:00  43.3876  68.0  37.0668  49.0  36.6285

In [177]:
calcMAE(df_res)

30.2062415292


In [178]:
saveResults(df_res,fName='30_3.csv',delta=0)

На борде даёт результат 34.53507

In [180]:
df_res

y1    y1_pr     y2    y2_pr     y3    y3_pr  \
region date                                                                  
1075   2016-06-01 00:00:00   16.0  3.25548   12.0  6.02036    3.0   8.9892   
       2016-06-01 01:00:00   12.0  9.24819    3.0  10.4773    0.0  13.1066   
       2016-06-01 02:00:00    3.0   10.849    0.0  12.1857    3.0  13.8932   
       2016-06-01 03:00:00    0.0  11.5391    3.0  12.2334   16.0   18.417   
       2016-06-01 04:00:00    3.0  6.60129   16.0   14.326   25.0  25.8311   
       2016-06-01 05:00:00   16.0  10.6712   25.0  22.7929   55.0  41.0749   
       2016-06-01 06:00:00   25.0  19.1439   55.0  38.0392   68.0  33.1435   
       2016-06-01 07:00:00   55.0  39.2594   68.0  32.0472   49.0  28.9672   
       2016-06-01 08:00:00   68.0  30.6522   49.0  25.7355   60.0  40.6361   
       2016-06-01 09:00:00   49.0   36.933   60.0  48.2667   84.0  57.1907   
       2016-06-01 10:00:00   60.0   62.098   84.0  67.9435   90.0  65.3768   
       2016-06-01 11:00:00   84.0   64.925   90.0   64.453  104.0  62.7008   
       2016-06-01 12:00:00   90.0  64.9653  104.0  65.5366  111.0  56.5799   
       2016-06-01 13:00:00  104.0  74.7012  111.0  65.6324   77.0   44.039   
       2016-06-01 14:00:00  111.0  69.7518   77.0  50.0746   78.0  55.2618   
       2016-06-01 15:00:00   77.0  59.1733   78.0  64.8397   94.0  71.9465   
       2016-06-01 16:00:00   78.0  81.8268   94.0  85.5371  130.0  74.3602   
       2016-06-01 17:00:00   94.0  81.4998  130.0  72.9972   97.0  65.2069   
       2016-06-01 18:00:00  130.0  68.3498   97.0  64.1938  122.0  67.1627   
       2016-06-01 19:00:00   97.0    70.97  122.0  74.1235  114.0  59.4589   
       2016-06-01 20:00:00  122.0  92.8396  114.0  74.5609   75.0  55.3038   
       2016-06-01 21:00:00  114.0  67.5227   75.0  51.1409   44.0  36.6989   
       2016-06-01 22:00:00   75.0  68.6853   44.0  49.0459   28.0  32.7874   
       2016-06-01 23:00:00   44.0  59.0333   28.0  40.7598   11.0  24.4272   
       2016-06-02 00:00:00   28.0  37.9644   11.0  24.8358    4.0  16.2447   
       2016-06-02 01:00:00   11.0  17.5797    4.0  13.4942    4.0  17.7459   
       2016-06-02 02:00:00    4.0   19.045    4.0  20.2314    6.0  21.3738   
       2016-06-02 03:00:00    4.0  13.0775    6.0  18.1316   15.0  32.0069   
       2016-06-02 04:00:00    6.0  15.0167   15.0  29.3497   28.0  52.1654   
       2016-06-02 05:00:00   15.0  30.6549   28.0  51.5558   78.0  79.2902   
...                           ...      ...    ...      ...    ...      ...   
2168   2016-06-29 18:00:00    0.0  43.4335    0.0  50.8088    1.0  61.8217   
       2016-06-29 19:00:00    0.0  43.2533    1.0  59.3925    1.0  67.1775   
       2016-06-29 20:00:00    1.0  51.4882    1.0  64.7088    0.0  51.7993   
       2016-06-29 21:00:00    1.0  55.7363    0.0  49.5137    0.0  37.0064   
       2016-06-29 22:00:00    0.0   38.193    0.0  33.8556    2.0  11.0335   
       2016-06-29 23:00:00    0.0  23.6171    2.0  7.53399    1.0        0   
       2016-06-30 00:00:00    2.0        0    1.0        0    0.0        0   
       2016-06-30 01:00:00    1.0        0    0.0        0    0.0        0   
       2016-06-30 02:00:00    0.0        0    0.0        0    0.0  27.0317   
       2016-06-30 03:00:00    0.0        0    0.0   25.999    8.0  31.5757   
       2016-06-30 04:00:00    0.0   29.335    8.0  30.6081    3.0  25.7768   
       2016-06-30 05:00:00    8.0  26.4401    3.0   24.274    2.0  22.9105   
       2016-06-30 06:00:00    3.0  15.9714    2.0  18.3918    1.0  21.3671   
       2016-06-30 07:00:00    2.0  12.7483    1.0  17.7635    1.0  18.3213   
       2016-06-30 08:00:00    1.0  13.0113    1.0  15.2142    0.0  12.8184   
       2016-06-30 09:00:00    1.0  12.1066    0.0  10.4505    2.0  22.4493   
       2016-06-30 10:00:00    0.0  9.54663    2.0   20.904    0.0  33.6265   
       2016-06-30 11:00:00    2.0  22.6647    0.0  33.5573    1.0  37.3079   
       2016-06-30 12:00:00    0.0  37.4368    1.0  39